In [ ]:
'''
chapter6 單純貝氏(詩詞分類)
    -資料集(李白、杜甫、白居易的詩)
    -目標(李白、杜甫、白居易)
    -建立詞向量
    -KNN試試看
    -貝氏小結論
'''

In [1]:
import pandas as pd
df = pd.read_csv("poem_train.csv", encoding='utf-8')

In [2]:
df['作者'] = df['作者'].astype('category')
#查看屬性改變  df.dtypes

In [3]:
#使用cat.cattegories得到所有類別，再用使cat.codes轉換成整數
save_map = {}

In [4]:
save_map = {cat:df['作者'].cat.categories.get_loc(cat) for cat in df['作者']}
save_map   

{'李白': 0, '杜甫': 1, '白居易': 2}

In [5]:
import jieba
def split_poem(poem):
    return " ".join(jieba.cut(poem))

In [6]:
def process_poems(df):
    df["內容"] = df["內容"].apply(split_poem)
    df["內容"] = df["內容"].str.replace("\n","")
    df["內容"] = df["內容"].str.replace("\r","")
    df = df.drop(["詩名"], axis = 1)
    df["作者"] = df["作者"].astype("category")
    return df

In [7]:
#建立詞向量
import jieba
df = process_poems(df)
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\keneau\AppData\Local\Temp\jieba.cache
Loading model cost 0.979 seconds.
Prefix dict has been built succesfully.


,作者,內容
0,李白,平林 漠漠 煙如織 ， 寒山 一帶 傷心 碧 。 暝 色入 高樓 ， 有人 樓上 愁 。 ...
1,李白,青天 有 月 來 幾時 ， 我今 停杯 一問 之 ： 人攀 明月 不可 得 ， 月行 卻 與...
2,李白,燕草 如碧絲 ， 秦桑低 綠枝 。 當君 懷歸日 ， 是 妾 斷腸時 。 春風 不 相識 ，...
3,李白,誰 家玉笛 暗飛聲 ， 散 入春 風滿 洛城 。 此 夜曲 中聞折 柳 ， 何人 不起 故園情 。
4,李白,西上 蓮 花山 ， 迢迢 見 明星 。 ( 西 上 一作 ： 西 嶽 ) 素 手把 芙蓉...


In [8]:
df['作者'] = df['作者'].cat.codes

In [9]:
test_df =pd.read_csv("poem_test.csv")
test_df = process_poems(test_df)
test_df.head()

,作者,內容
0,李白,日照 香爐生 紫煙 ， 遙看 瀑布 掛 前川 。 飛流 直下 三千尺 ， 疑是 銀河 落九天 。
1,李白,朝辭 白帝 彩雲間 ， 千裡 江陵 一日 還 。 兩岸 猿聲 啼 不住 ， 輕舟 已過 萬 ...
2,李白,李白 乘舟 將欲行 ， 忽聞 岸上 踏歌 聲 。 桃花潭水 深 千尺 ， 不及 汪倫送 我情 。
3,李白,故人 西辭黃鶴樓 ， 煙花 三月 下揚州 。 孤帆 遠影 碧空 儘 ， 唯見長 江天 際流 。
4,李白,危樓 高 百尺 ， 手可摘 星辰 。 不敢 高聲語 ， 恐驚 天上 人 。


In [10]:
test_df['作者'] = test_df['作者'].replace(save_map)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
vec = TfidfVectorizer()
bag = vec.fit_transform(df['內容'])
#alpha可是結果調整
clf = MultinomialNB(alpha = 0.001)
clf.fit(bag, df['作者'])

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [12]:
from sklearn.metrics import accuracy_score
test_bag = vec.transform(test_df['內容'])
predict = clf.predict(test_bag)
print("預測:",list(predict))
print("正確標籤:",list(test_df['作者']))
print('Naive-Bayes 正確率',accuracy_score(test_df['作者'],predict)*100,"%")

預測: [0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Naive-Bayes 正確率 80.0 %


In [14]:
#KNN試試看
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors = 3)
clf.fit(bag, df['作者'])
predict = clf.predict(test_bag)
print("預測:",list(predict))
print("正確標籤:",list(test_df['作者']))
print('KNN 正確率',accuracy_score(test_df['作者'],predict)*100,"%")

預測: [0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, 2, 2, 0, 1, 1, 1, 2, 2, 2, 1, 0, 0, 0, 0, 2, 2, 1]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
KNN 正確率 50.0 %


In [ ]:
#結論:對文字處理，貝氏也很適合
#上次新聞因為向量較長訓練自料較多也較好的效果，而這次詩詞是比較短的向量，這邊可以看出資料較少時機率算法不錯